<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span><ul class="toc-item"><li><span><a href="#Java.util-implementation" data-toc-modified-id="Java.util-implementation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Java.util implementation</a></span></li></ul></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example</a></span></li></ul></div>

# Observer

## Intro

The Observer pattern defines a one to many dependency between objects that allows an object to notify its dependents of a change in its state. This should happen automatically. This needs to happen whenever an event occurs and we don't want to change the object broadcasting.

A good real world example of this is a radio transmitter. The transmitter doesn't change, but its broadcast is received by a radio device that then interprets it and does what it needs to do.

The pattern is designed to decouple the notifiers and notifiees.

The pattern consists of observers and subjects. The observers register themselves to a subject and are automatically notified when a change on the subject happens. If they no longer wish to see updates, they can unregister from the subject. This design allows observers to receive updates only from subjects they care about and allows flexibility in how observers and subjects interact

Advantages to the pattern are:
* Produces loose coupling between the subjects and observers
* The subject only knows about the observer via the interface (removes state leakage)
* New observers can be added at any time making this adaptable to change
* Never need to change the subject to communicate the observer
* Can reuse subjects and observers independently
* changes to observers and subjects do not affect each other

### Java.util implementation

Java has its own version of the observable pattern baked into the standard util package. It's similar to our subject and observer interfaces but has a lot of functionality out of the box, removing the need to implement this ourselves. It allows for push or pull updates to observers (push subject sends, pull observer requests)

The Observable class and the Observer interface make up the pattern. The Observable class plays the role of the subject.

Instead of the methods attach, detach and notify Java uses addObserver, deleteObserver and notifyObservers.

For an object to become an observer, it needs to implement the observer interface. Then, the subject (Observable instance) should then call addObserver() to have this appear as an observer. To remove, we use deleteObserver()



In order to send notifications via this implementation though, we need to extend the super class Observable. We'd then need to call the setChanged method to show the state changed. Only then can we notify using the appropriate methods. This is much heavier initially, but can be managed with some delegation and encapsulation.

The observer (real observer here) needs to implement the update method (this is in the interface) in order to receive updates from the Observable. It is here that we determine the style of update, either push or pull. There are 2 methods on observables for updates, one taking an object, the other not. These are for push and pull updates respectively.

for the pull style, this is why the setChanged method needs to be called before notifications are sent. If this does not happen, not notification will happen as effectively, nothing has changed.

The Java implementation is not really reusable and requires us to build the system with the pattern in mind. With there not being an interface (like the cloneable sitch) and *having* to extend the super class, the implementation is brittle and difficult to work into new systems. It's also really limited in what it can do and does not have most of the advantages the pattern normally has. We might see this in the wild, but it's lowkey trash.

## Implementation

In [93]:
interface Observer {
    void update();
    void setSubject(Subject sub);
}

In [94]:
interface Subject {
    void registerObs(Observer obs);
    void unregisterObs(Observer obs);
    void notifyObs();
    Object getUpdate(Observer obs);
}

In [95]:
class MyTopic implements Subject {
    private List<Observer> observers = new ArrayList<>();
    private String message;
    private boolean changed;
    
    @Override
    public void registerObs(Observer obs)
    {
        if (observers.contains(obs)) {
            return;
        }
        
        observers.add(obs);
    }
    
    @Override
    public void unregisterObs(Observer obs)
    {       
        observers.remove(obs);
    }
    
    @Override
    public void notifyObs()
    {        
        if (!changed) {
            return;
        }
        
        // Stand in for a lock on the resource
        List<Observer> localList = new ArrayList<>(this.observers);
        this.changed = false;
        
        for (Observer obs : localList) {
            obs.update();
        }
    }
    
    public Object getUpdate(Observer obs)
    {
        return this.message;
    }
    
    public void postMessage(String msg)
    {
        System.out.println("Posted this message: " + msg);
        this.message = msg;
        this.changed = true;
        notifyObs();
    }
}

In [96]:
class MySubscriber implements Observer {
    private String name;
    private Subject topic;
    
    public MySubscriber(String name)
    {
        this.name = name;
    }
    
    // Does push and pull updates
    public void update()
    {
        String msg = (String) topic.getUpdate(this);
        
        if (msg == null) {
            System.out.println(name + ":: no new message");
        } else {
            System.out.println(name + ":: Consuming message " + msg);
        }
    }
    
    public void setSubject(Subject sub)
    {
        this.topic = sub;
    }
}

In [97]:
MyTopic myTopic = new MyTopic();

Observer ob1 = new MySubscriber("Harry");
Observer ob2 = new MySubscriber("Thomas");
Observer ob3 = new MySubscriber("Percy");

ob1.setSubject(myTopic);
ob2.setSubject(myTopic);
ob3.setSubject(myTopic);

In [98]:
myTopic.registerObs(ob1);
myTopic.registerObs(ob2);
myTopic.registerObs(ob3);

In [99]:
ob1.update();

Harry:: no new message


In [100]:
myTopic.postMessage("Hello world");

Posted this message: Hello world
Harry:: Consuming message Hello world
Thomas:: Consuming message Hello world
Percy:: Consuming message Hello world


## Example

In [101]:
interface Observer {
    void update();
    void setSubject(Subject sub);
}

In [ ]:
interface Subject {
    void registerObs(Observer obs);
    void unregisterObs(Observer obs);
    void notifyObs();
    Object getUpdate(Observer obs);
}

In [106]:
class CricketData {
    private int score;
    
    public int getScore()
    {
        return score;
    }
    
    public void setScore(int num)
    {
        score += num;
    }
}

In [111]:
class Punter implements Observer {
    private Subject cricketGame;

    public void update()
    {
        CricketData data = (CricketData) this.cricketGame.getUpdate(this);
        
        if (data == null) {
            System.out.println("No scores yet");
        } else {
            System.out.println("Score is " + data.getScore());
        }
    }
    
    public void setSubject(Subject sub)
    {
        this.cricketGame = sub;
    }
}

In [115]:
class CricketGame implements Subject {
    private List<Observer> attendees = new ArrayList<>();
    private boolean changed;
    private CricketData cricketData = new CricketData();
    
    @Override
    public void registerObs(Observer obs)
    {
        if (attendees.contains(obs)) {
            return;
        }
        
        attendees.add(obs);
        obs.setSubject(this);
    }
    
    @Override
    public void unregisterObs(Observer obs)
    {       
        attendees.remove(obs);
    }
    
    @Override
    public void notifyObs()
    {        
        if (!changed) {
            return;
        }
        
        // Stand in for a lock on the resource
        List<Observer> localList = new ArrayList<>(this.attendees);
        this.changed = false;
        
        for (Observer obs : localList) {
            obs.update();
        }
    }
    
    public Object getUpdate(Observer obs)
    {
        return this.cricketData;
    }
    
    public void postMessage(CricketData data)
    {
        this.cricketData = data;
        System.out.println("Sending score: " + cricketData.getScore());
        this.changed = true;
        notifyObs();
    }
}

In [116]:
Observer w1 = new Punter();
Observer w2 = new Punter();
Observer w3 = new Punter();

In [117]:
CricketGame cricketGame = new CricketGame();

In [120]:
cricketGame.registerObs(w1);
cricketGame.registerObs(w2);
cricketGame.registerObs(w3);

In [121]:
CricketData data = new CricketData();
data.setScore(56);

In [122]:
cricketGame.postMessage(data);

Sending score: 56
Score is 56
Score is 56
Score is 56
